In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from unidecode import unidecode

df = pd.read_csv('Data/data.csv')
df = df.drop(columns=['archivo'])

In [ ]:
def normalize_str_col(df, col):
    df[col] = df[col].apply(lambda x: unidecode(x) if isinstance(x, str) else x)
    df[col] = df[col].str.lower()
    df[col] = df[col].replace('|', np.nan)
    df = df.dropna().copy()
    df[col] = df[col].apply(lambda x: re.sub(r'\b(de|la|el|y)\b', '', x))  # Eliminar stop words comunes
    df[col] = df[col].apply(lambda x: re.sub(r'[0-9\.]', '', unidecode(x)) if isinstance(x, str) else x) # Eliminar números y puntos
    df[col] = df[col].apply(lambda x: re.sub(r'\s+', ' ', x).strip())  # Eliminar espacios extra
    df[col] = df[col].replace('', np.nan)
    df[col] = df[col].str.strip() # Eliminar espacios al principio y al final
    df = df.dropna()
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    df = df.copy()
    return df

In [ ]:
data = normalize_str_col(df, 'Descripcion')
data.shape

In [ ]:
df_codes = pd.read_excel('Lista_codigos.xlsx')
code_list = df_codes['Codigo'].tolist()

filtro1 = data['Codigo'].isin(code_list)
data = data[filtro1].copy()
data = data.reset_index(drop=True)
data.shape

## Analizar Grupos

In [ ]:
codes_groped = data.groupby(['Codigo']).agg({
    'Descripcion': 'count'
}).reset_index()
codes_groped = codes_groped.rename(columns={'Descripcion': 'Cantidad'})
codes_groped['Especialidad'] = codes_groped['Codigo'].map(df_codes.set_index('Codigo')['Especialidad'])
codes_groped = codes_groped.sort_values(by='Cantidad', ascending=False)
codes_groped

In [ ]:
codes_groped.groupby('Especialidad').agg({
    'Cantidad': 'sum'
}).reset_index().sort_values(by='Cantidad', ascending=False)


In [ ]:
black_list = ['ELEV', 'S-E', 'IEP', 'ISP', 'EQC']
filtro2 = ~data['Codigo'].isin(black_list)
data = data[filtro2].copy()
data.shape

In [ ]:
data['Especialidad'] = data['Codigo'].map(df_codes.set_index('Codigo')['Especialidad'])
data = data.reset_index(drop=True)
data.to_csv('Data/data_clean.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
data['Codigo'].value_counts().plot(kind='bar')
plt.show()
